In [1]:
from time import time
import tensorflow as tf
import multiprocessing as mp
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import CSVLogger, EarlyStopping, LambdaCallback, TensorBoard
import matplotlib.pyplot as plt
import random

Using TensorFlow backend.


In [2]:
def get_model(neurons_in_layer, number_of_layers=2, lr=0.1, activation_function='relu', dropout_rates=0.2):
    """
    Design the model with Multilevel Perceptrons
    :return:
    """
    model = Sequential()

    model.add(Dense(neurons_in_layer, activation=activation_function, input_shape=(3072,)))
    model.add(Dropout(dropout_rates))
    for i in range(number_of_layers-1):
        model.add(Dense(neurons_in_layer, activation=activation_function))
        model.add(Dropout(dropout_rates))

    model.add(Dense(num_classes, activation='softmax'))

    sgd = SGD(lr=lr, decay=1e-6, nesterov=True)

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.summary()
    return model

In [3]:
def get_preprocessed_data(x_train, x_test, y_train, y_test):
    """
    Preprocess images. convert them into vector.
    :return:
    """
    x_train = x_train.reshape(50000, 3072)
    x_test = x_test.reshape(10000, 3072)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)

    return x_train, x_test, y_train, y_test

In [4]:
def run(epochs, batch_size, neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates,
        x_train, x_test, y_train, y_test, callbacks=[]):
    x_train, x_test, y_train, y_test = get_preprocessed_data(x_train, x_test, y_train, y_test)
    model = get_model(neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates)
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        callbacks=callbacks)

    scores = model.evaluate(x_test, y_test, verbose=0)
    print(scores)
    return model

In [5]:
def run1(epochs, batch_size, model, x_train, x_test, y_train, y_test, callbacks=[]):
    x_train, x_test, y_train, y_test = get_preprocessed_data(x_train, x_test, y_train, y_test)
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        callbacks=callbacks)

    scores = model.evaluate(x_test, y_test, verbose=0)
    print(scores)
    return model

In [6]:
def save_model_prediction_graph(epoch, logs):
    prediction_probs = model.predict_proba(grid_2d, batch_size=32, verbose=0)
    plt.figure(figsize=(10,10))
    sns.set_style("whitegrid")
    plt.title('Classification with KERAS - epoch: ' + makeIndexOfLength(epoch, 3), fontsize=20)
    plt.xlabel('X', fontsize=15)
    plt.ylabel('Y', fontsize=15)
    plt.contourf(X, Y, prediction_probs.reshape(100, 100), alpha = 0.7, cmap=cm.Spectral)
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train.ravel(), s=50, cmap=plt.cm.Spectral, edgecolors='black')
    plt.savefig("./" + OUTPUT_DIR + "/keras" + makeIndexOfLength(epoch, 3) + ".png")
    plt.close()

In [7]:
tensorboard = TensorBoard(log_dir="logs/callbacks/{}".format(time()))

num_classes = 10

if K.backend() == 'tensorflow':
    K.set_image_dim_ordering("th")

core_num = mp.cpu_count()
config = tf.ConfigProto(
    inter_op_parallelism_threads=core_num,
    intra_op_parallelism_threads=core_num)

sess = tf.Session(config=config)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Declare variables
epochs = 15
batch_size = 32

neurons_in_layer = 512
number_of_layers = 2
lr = 0.1
activation_function = 'relu'
dropout_rates = 0.2


x_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [ ]:
# boundary of the graph
GRID_X_START = -1.5
GRID_X_END = 1.5
GRID_Y_START = -1.5
GRID_Y_END = 1.5
# output directory (the folder must be created on the drive)
OUTPUT_DIR = "classification_vizualizations"

grid = np.mgrid[GRID_X_START:GRID_X_END:100j,GRID_X_START:GRID_Y_END:100j]
grid_2d = grid.reshape(2, -1).T
X, Y = grid
acc_history = []
loss_history = []
model = get_model(neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates)
testmodelcb = LambdaCallback(on_epoch_end=save_model_prediction_graph)
callbacks = [testmodelcb]
run1(epochs, batch_size, model, x_train, x_test, y_train, y_test, callbacks)

In [ ]:
csv_logger = CSVLogger('training.log')
callbacks = [csv_logger, tensorboard]
run(epochs, batch_size, neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates, x_train, x_test, y_train, y_test, callbacks)

In [ ]:
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
callbacks = [earlystopper]
run(epochs, batch_size, neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates, x_train, x_test, y_train, y_test, callbacks)

In [12]:
batch_print_callback = LambdaCallback(
    on_batch_begin=lambda batch,logs: print(batch))

callbacks = [batch_print_callback]
epochs = 3
model = run(epochs, batch_size, neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates, x_train, x_test, y_train, y_test, callbacks)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/3
0
   32/50000 [..............................] - ETA: 8:52 - loss: 2.284

 7904/50000 [===>..........................] - ETA: 24s - loss: 2.2305 - acc: 0.1788247
248
249
250
 8032/50000 [===>..........................] - ETA: 24s - loss: 2.2266 - acc: 0.1802251
252
253
 8128/50000 [===>..........................] - ETA: 24s - loss: 2.2243 - acc: 0.1810254
255
256
 8224/50000 [===>..........................] - ETA: 24s - loss: 2.2211 - acc: 0.1815257
258
259
 8320/50000 [===>..........................] - ETA: 24s - loss: 2.2207 - acc: 0.1811260
261
262
 8416/50000 [====>.........................] - ETA: 24s - loss: 2.2203 - acc: 0.1811263
264
265
 8512/50000 [====>.........................] - ETA: 24s - loss: 2.2189 - acc: 0.1820266
267
268
 8608/50000 [====>.........................] - ETA: 24s - loss: 2.2162 - acc: 0.1833269
270
271
 8704/50000 [====>.........................] - ETA: 24s - loss: 2.2144 - acc: 0.1835272
273
274
 8800/50000 [====>.........................] - ETA: 24s - loss: 2.2134 - acc: 0.1839275
276
277
 8896/50000 [====>..................

16992/50000 [=========>....................] - ETA: 18s - loss: 2.1296 - acc: 0.2199531
532
533
534
17120/50000 [=========>....................] - ETA: 18s - loss: 2.1285 - acc: 0.2202535
536
537
17216/50000 [=========>....................] - ETA: 18s - loss: 2.1278 - acc: 0.2206538
539
540
541
17344/50000 [=========>....................] - ETA: 18s - loss: 2.1264 - acc: 0.2209542
543
544
17440/50000 [=========>....................] - ETA: 18s - loss: 2.1256 - acc: 0.2213545
546
547
548
17568/50000 [=========>....................] - ETA: 17s - loss: 2.1241 - acc: 0.2220549
550
551
17664/50000 [=========>....................] - ETA: 17s - loss: 2.1226 - acc: 0.2223552
553
554
17760/50000 [=========>....................] - ETA: 17s - loss: 2.1211 - acc: 0.2230555
556
557
17856/50000 [=========>....................] - ETA: 17s - loss: 2.1193 - acc: 0.2232558
559
560
17952/50000 [=========>....................] - ETA: 17s - loss: 2.1185 - acc: 0.2238561
562
563
564
18080/50000 [=========>.

816
817
26176/50000 [==============>...............] - ETA: 12s - loss: 2.0664 - acc: 0.2420818
819
820
26272/50000 [==============>...............] - ETA: 12s - loss: 2.0661 - acc: 0.2419821
822
823
824
26400/50000 [==============>...............] - ETA: 12s - loss: 2.0657 - acc: 0.2420825
826
827
26496/50000 [==============>...............] - ETA: 12s - loss: 2.0652 - acc: 0.2421828
829
830
26592/50000 [==============>...............] - ETA: 12s - loss: 2.0644 - acc: 0.2426831
832
833
834
26720/50000 [===============>..............] - ETA: 12s - loss: 2.0644 - acc: 0.2428835
836
837
26816/50000 [===============>..............] - ETA: 12s - loss: 2.0644 - acc: 0.2430838
839
840
26912/50000 [===============>..............] - ETA: 12s - loss: 2.0642 - acc: 0.2430841
842
843
27008/50000 [===============>..............] - ETA: 12s - loss: 2.0641 - acc: 0.2430844
845
846
847
27136/50000 [===============>..............] - ETA: 12s - loss: 2.0627 - acc: 0.2432848
849
850
27232/50000 [=======

35072/50000 [====================>.........] - ETA: 8s - loss: 2.0313 - acc: 0.25691096
1097
1098
35168/50000 [====================>.........] - ETA: 8s - loss: 2.0309 - acc: 0.25711099
1100
1101
35264/50000 [====================>.........] - ETA: 7s - loss: 2.0301 - acc: 0.25751102
1103
1104
35360/50000 [====================>.........] - ETA: 7s - loss: 2.0295 - acc: 0.25771105
1106
1107
35456/50000 [====================>.........] - ETA: 7s - loss: 2.0293 - acc: 0.25801108
1109
1110
35552/50000 [====================>.........] - ETA: 7s - loss: 2.0285 - acc: 0.25811111
1112
1113
35648/50000 [====================>.........] - ETA: 7s - loss: 2.0284 - acc: 0.25811114
1115
1116
35744/50000 [====================>.........] - ETA: 7s - loss: 2.0280 - acc: 0.25821117
1118
1119
35840/50000 [====================>.........] - ETA: 7s - loss: 2.0283 - acc: 0.25811120
1121
1122
35936/50000 [====================>.........] - ETA: 7s - loss: 2.0276 - acc: 0.25831123
1124
1125
36032/50000 [=======

...

 3520/50000 [=>............................] - ETA: 16s - loss: 1.7520 - acc: 0.3662110
111
112
113
114
 3680/50000 [=>............................] - ETA: 16s - loss: 1.7510 - acc: 0.3655115
116
117
118
 3808/50000 [=>............................] - ETA: 16s - loss: 1.7521 - acc: 0.3653119
120
121
122
 3936/50000 [=>............................] - ETA: 16s - loss: 1.7497 - acc: 0.3671123
124
125
126
 4064/50000 [=>............................] - ETA: 16s - loss: 1.7461 - acc: 0.3691127
128
129
130
131
 4224/50000 [=>............................] - ETA: 16s - loss: 1.7447 - acc: 0.3707132
133
134
 4320/50000 [=>............................] - ETA: 16s - loss: 1.7394 - acc: 0.3715135
136
137
 4416/50000 [=>............................] - ETA: 17s - loss: 1.7413 - acc: 0.3714138
139
140
 4512/50000 [=>............................] - ETA: 17s - loss: 1.7418 - acc: 0.3715141
142
143
 4608/50000 [=>............................] - ETA: 17s - loss: 1.7440 - acc: 0.3715144
145
146
 4704/50000 

12576/50000 [======>.......................] - ETA: 19s - loss: 1.7501 - acc: 0.3709393
394
12640/50000 [======>.......................] - ETA: 19s - loss: 1.7497 - acc: 0.3714395
396
12736/50000 [======>.......................] - ETA: 19s - loss: 1.7497 - acc: 0.3715398
399
400
12832/50000 [======>.......................] - ETA: 19s - loss: 1.7502 - acc: 0.3709401
402
403
12960/50000 [======>.......................] - ETA: 19s - loss: 1.7499 - acc: 0.3711405
406
13024/50000 [======>.......................] - ETA: 19s - loss: 1.7501 - acc: 0.3710407
408
13088/50000 [======>.......................] - ETA: 19s - loss: 1.7497 - acc: 0.3712409
410
411
412
13216/50000 [======>.......................] - ETA: 19s - loss: 1.7506 - acc: 0.3711413
414
415
416
13344/50000 [=======>......................] - ETA: 19s - loss: 1.7511 - acc: 0.3711417
418
419
420
421
13504/50000 [=======>......................] - ETA: 19s - loss: 1.7522 - acc: 0.3706422
423
424
425
426
13664/50000 [=======>...........

23808/50000 [=============>................] - ETA: 12s - loss: 1.7507 - acc: 0.3713744
745
746
747
23936/50000 [=============>................] - ETA: 12s - loss: 1.7509 - acc: 0.3711748
749
750
24032/50000 [=============>................] - ETA: 12s - loss: 1.7516 - acc: 0.3712751
752
24096/50000 [=============>................] - ETA: 12s - loss: 1.7517 - acc: 0.3711753
754
24160/50000 [=============>................] - ETA: 12s - loss: 1.7518 - acc: 0.3710755
756
757
24256/50000 [=============>................] - ETA: 12s - loss: 1.7519 - acc: 0.3706758
759
760
761
24384/50000 [=============>................] - ETA: 11s - loss: 1.7522 - acc: 0.3704762
763
764
765
766
24544/50000 [=============>................] - ETA: 11s - loss: 1.7521 - acc: 0.3704767
768
769
770
771
24704/50000 [=============>................] - ETA: 11s - loss: 1.7518 - acc: 0.3708772
773
774
775
776
24864/50000 [=============>................] - ETA: 11s - loss: 1.7503 - acc: 0.3712777
778
779
780
24992/50000 

34880/50000 [===================>..........] - ETA: 6s - loss: 1.7463 - acc: 0.37231090
1091
1092
1093
35008/50000 [====================>.........] - ETA: 6s - loss: 1.7466 - acc: 0.37211094
1095
1096
1097
35136/50000 [====================>.........] - ETA: 6s - loss: 1.7465 - acc: 0.37201098
1099
1100
35232/50000 [====================>.........] - ETA: 6s - loss: 1.7464 - acc: 0.37191101
1102
1103
35328/50000 [====================>.........] - ETA: 6s - loss: 1.7464 - acc: 0.37201104
1105
1106
35424/50000 [====================>.........] - ETA: 6s - loss: 1.7460 - acc: 0.37211107
1108
1109
35520/50000 [====================>.........] - ETA: 6s - loss: 1.7462 - acc: 0.37211110
1111
1112
1113
1114
35680/50000 [====================>.........] - ETA: 6s - loss: 1.7461 - acc: 0.37211115
1116
1117
1118
1119
35840/50000 [====================>.........] - ETA: 6s - loss: 1.7463 - acc: 0.37211120
1121
1122
1123
1124
36000/50000 [====================>.........] - ETA: 6s - loss: 1.7463 - acc: 0

44864/50000 [=========================>....] - ETA: 2s - loss: 1.7430 - acc: 0.37151402
1403
1404
1405
44992/50000 [=========================>....] - ETA: 2s - loss: 1.7426 - acc: 0.37161406
1407
1408
1409
1410
45152/50000 [==========================>...] - ETA: 2s - loss: 1.7428 - acc: 0.37151411
1412
1413
1414
1415
45312/50000 [==========================>...] - ETA: 2s - loss: 1.7425 - acc: 0.37161416
1417
1418
1419
1420
45472/50000 [==========================>...] - ETA: 2s - loss: 1.7430 - acc: 0.37131421
1422
1423
1424
1425
45632/50000 [==========================>...] - ETA: 1s - loss: 1.7421 - acc: 0.37161426
1427
1428
1429
1430
45792/50000 [==========================>...] - ETA: 1s - loss: 1.7425 - acc: 0.37141431
1432
1433
1434
1435
45952/50000 [==========================>...] - ETA: 1s - loss: 1.7427 - acc: 0.37141436
1437
1438
1439
1440
46112/50000 [==========================>...] - ETA: 1s - loss: 1.7431 - acc: 0.37151441
1442
1443
1444
1445
46272/50000 [====================

In [11]:
cifar10_labels = np.array([
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'])
def convertCIFER10Data(image):
    img = image.astype('float32')
    img /= 255
    c = np.zeros(32*32*3).reshape((1,32,32,3))
    c[0] = img
    return c

plt.figure(figsize=(16,16))

right = 0
mistake = 0

for i in range(100):
    index = random.randint(0, x_test.shape[0])
    image = x_test[index]
#     data = convertCIFER10Data(image)
    data = image
#     plt.subplot(10, 10, i+1)
#     plt.imshow(image)
#     plt.axis('off')

    ret = model.predict(data, batch_size=1) 
    #print(ret)

    bestnum = 0.0
    bestclass = 0
    for n in [0,1,2,3,4,5,6,7,8,9]:
        if bestnum < ret[0][n]:
            bestnum = ret[0][n]
            bestclass = n

    if y_test[index] == bestclass:
        plt.title(cifar10_labels[bestclass])
        right += 1
    else:
        plt.title(cifar10_labels[bestclass] + "!=" + cifar10_labels[y_test[index][0]], color='#ff0000')
        mistake += 1
                                                                   
plt.show()
print("The number of correct answers:", right)
print("The number of mistake:", mistake)
print("A correct answer rate:", right/(mistake + right)*100, '%')


ValueError: Error when checking input: expected dense_1_input to have 2 dimensions, but got array with shape (3, 32, 32)

<Figure size 1152x1152 with 0 Axes>